In [5]:
import numpy as np
import pandas as pd
import cv2
import dlib
import imutils
import os
import joblib
import cvlib as cv
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.models import load_model

In [77]:
!pip install numpy
!pip install pandas
!pip install cvlib
!pip install opencv-python
!pip install imutils
!pip install tensorflow
!pip install joblib
!pip install dlib-19.24.99-cp312-cp312-win_amd64.whl

     ---------------------------------------- 0.0/13.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/13.1 MB ? eta -:--:--
      --------------------------------------- 0.3/13.1 MB 4.3 MB/s eta 0:00:03
     --- ------------------------------------ 1.1/13.1 MB 9.8 MB/s eta 0:00:02
     ------ --------------------------------- 2.1/13.1 MB 14.8 MB/s eta 0:00:01
     --------- ------------------------------ 3.2/13.1 MB 17.0 MB/s eta 0:00:01
     ------------- -------------------------- 4.5/13.1 MB 19.0 MB/s eta 0:00:01
     --------------------- ------------------ 7.1/13.1 MB 25.2 MB/s eta 0:00:01
     ------------------------------ --------- 9.8/13.1 MB 28.6 MB/s eta 0:00:01
     --------------------------------------  13.1/13.1 MB 54.4 MB/s eta 0:00:01
     --------------------------------------- 13.1/13.1 MB 43.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): sta

In [7]:
# function for gender recognition 

def gender_recognition(image_path):
    #preprocessing
    img_size = (128, 128)  # Target size for images
    front_img = load_img(image_path, target_size=img_size)
    front_img = img_to_array(front_img)
    front_img = np.expand_dims(front_img, axis=0) 

    #loading our pretrained model
    model = load_model("./pretrained_models/front_model.h5")
    # Make a prediction
    prediction = model.predict(front_img)
    
    # Interpret the prediction
    if prediction[0][0] > 0.5:
        return "Female"
    else:
        return "Male"

In [9]:
#extracting facial features from the image using a pretrained model
# Load the pre-trained facial landmarks model
landmark_model_path = "./pretrained_models/shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(landmark_model_path)

def extract_landmarks(image_path):
    # Read the image
    image = cv2.imread(image_path)
    image = imutils.resize(image, width=600)

    if image is None:
        print("Hello")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = detector(gray)
    if len(faces) == 0:
        print("No faces detected!")
        return None

    # Assume the first face detected is the target face
    landmarks = predictor(gray, faces[0])

    # Extract (x, y) coordinates of the landmarks
    points = []
    for n in range(68):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        points.append((x, y))

    return points

def compute_features(front_landmarks, side_landmarks=None):
    # Cheek-to-Jaw Width Ratio (CJWR)
    jaw_width = np.linalg.norm(np.array(front_landmarks[3]) - np.array(front_landmarks[13]))
    cheek_width = np.linalg.norm(np.array(front_landmarks[1]) - np.array(front_landmarks[15]))
    cjwr = cheek_width / jaw_width

    # Width to Upper Facial Height Ratio (WHR)
    width = np.linalg.norm(np.array(front_landmarks[1]) - np.array(front_landmarks[15]))
    upper_height = np.linalg.norm(np.array(front_landmarks[27]) - np.array(front_landmarks[8]))
    whr = width / upper_height

    # Perimeter to Area Ratio (PAR)
    contour_points = np.array(front_landmarks[:17])  # Jawline points
    perimeter = cv2.arcLength(contour_points, True)
    area = cv2.contourArea(contour_points)
    par = perimeter / area if area != 0 else None

    # Eye Size (ES)
    left_eye_width = np.linalg.norm(np.array(front_landmarks[36]) - np.array(front_landmarks[39]))
    left_eye_height = np.linalg.norm(np.array(front_landmarks[37]) - np.array(front_landmarks[41]))
    right_eye_width = np.linalg.norm(np.array(front_landmarks[42]) - np.array(front_landmarks[45]))
    right_eye_height = np.linalg.norm(np.array(front_landmarks[43]) - np.array(front_landmarks[47]))
    es = ((left_eye_width * left_eye_height) + (right_eye_width * right_eye_height)) / 2

    # Lower Face to Face Height Ratio (FW/FH)
    lower_face_height = np.linalg.norm(np.array(front_landmarks[33]) - np.array(front_landmarks[8]))
    face_height = np.linalg.norm(np.array(front_landmarks[27]) - np.array(front_landmarks[8]))
    fw_fh = lower_face_height / face_height

    # Mean Eyebrow Height (MEH)
    left_eyebrow_mean_height = np.mean([np.linalg.norm(np.array(front_landmarks[19]) - np.array(front_landmarks[37])),
                                        np.linalg.norm(np.array(front_landmarks[20]) - np.array(front_landmarks[38]))])
    right_eyebrow_mean_height = np.mean([np.linalg.norm(np.array(front_landmarks[23]) - np.array(front_landmarks[43])),
                                         np.linalg.norm(np.array(front_landmarks[24]) - np.array(front_landmarks[44]))])
    meh = (left_eyebrow_mean_height + right_eyebrow_mean_height) / 2

    return {
        "front_CJWR": cjwr,
        "front_WHR": whr,
        "front_PAR": par,
        "front_ES": es,
        "front_FW/FH": fw_fh,
        "front_MEH": meh
    }


In [11]:
#bmi classification
def bmi_class(bmi):
    if bmi<18.5:
        return "underweight"
    elif bmi<24.9:
        return "normal"
    elif bmi<29.9:
        return "overweight"
    elif bmi>=29.9:
        return "obesity"
    else:
        return "error"

In [13]:
model = joblib.load('models/linear_reg.pkl')
model2 = joblib.load('models/cat_reg.pkl')
model3 = joblib.load('models/linear_reg2.pkl')

In [17]:
path = './test_images/ritesh.jpg'

landmarks = extract_landmarks(path)
features = compute_features(landmarks)

df = pd.DataFrame([features])
sex = gender_recognition(path)
df['sex'] = 0 if sex=="Male" else 1 
# print(df['sex'])
print(sex)
y = model.predict(df)
y2 = model2.predict(df)
y3 = model3.predict(df)
print(y)
print(bmi_class(y))
print(y2)
print(bmi_class(y2))
print(y3)
print(bmi_class(y3))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
Male
[[25.02032039]]
overweight
[25.02868681]
overweight
[[24.2140293]]
normal
